In [1]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

import data_tools as dt


In [2]:
agenda_df = dt.get_agenda_items()
minutes_df = dt.get_minutes()
supplemental_df = dt.get_supplemental_docs()

No data found for 2021-01-14
No data found for 2022-03-17
No data found for 2022-10-13
No proper response found for 2018-05-23, doc_id 1, pages 2-2
No proper response found for 2018-05-23, doc_id 2, pages 3-3
No proper response found for 2018-07-26, doc_id 1, pages 2-2


In [3]:
"""
Check if each agenda item with a casenum is contained in the minutes
"""
df = agenda_df.loc[
    agenda_df['is_casenum'], 
    ['year', 'date', 'item_no', 'title', 'is_casenum']
].reset_index(drop=True)

df = df.merge(
    minutes_df[['year', 'date', 'item_no', 'project_result']],
    on = ['year', 'date', 'item_no'],
    how = 'left',
).reset_index(drop=True)

assert df['project_result'].isna().sum() == 0

In [4]:
"""
Check if all the council members in the voting data are correctly identified
"""
for idx, row in minutes_df.iterrows():
    for col in ['moved', 'seconded', 'ayes', 'nays', 'abstained', 'recused', 'absent']:
        names = [x.strip() for x in row[col].split(',')]
        for name in names:
            assert name in dt.MEMBERS

In [5]:
supplemental_df['support_or_oppose'].value_counts()

support_or_oppose
DEFINITELY OPPOSE                                                                                      2134
DEFINITELY SUPPORT                                                                                     1794
NOT RELEVANT                                                                                           1459
SOMEWHAT SUPPORT                                                                                        305
SOMEWHAT OPPOSE                                                                                         296
NEUTRAL                                                                                                 197
NOT RELEVANT (The document does not provide enough information to determine support or opposition.)       2
SOMEWHAT SUPPORT (for Items 6 and 7), SOMEWHAT OPPOSE (for Item 9)                                        1
Name: count, dtype: int64